# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("demo01.csv", encoding="latin1")

In [30]:
# Verificar os tipos de dados
df.dtypes


ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object

In [32]:
# Contar valores ausentes por coluna
df.isnull().sum()


ID                         0
CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_INCOME_TOTAL           0
NAME_INCOME_TYPE           0
NAME_EDUCATION_TYPE        0
NAME_FAMILY_STATUS         0
NAME_HOUSING_TYPE          0
DAYS_BIRTH                 0
DAYS_EMPLOYED              0
FLAG_MOBIL                 0
FLAG_WORK_PHONE            0
FLAG_PHONE                 0
FLAG_EMAIL                 0
OCCUPATION_TYPE        25120
CNT_FAM_MEMBERS            0
dtype: int64

In [34]:
print(df.columns.tolist())  # Mostra a lista de colunas exatas


['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS']


In [36]:
# Verificar e corrigir o nome das colunas
df.columns = df.columns.str.strip().str.lower()

# Confirmar se 'mau' existe antes de acessar
if 'mau' in df.columns:
    resposta_counts = df['mau'].value_counts()

    # Plotar a distribuição
    plt.figure(figsize=(6, 4))
    sns.barplot(x=resposta_counts.index, y=resposta_counts.values, palette="viridis")
    plt.title("Distribuição da Variável Resposta ('mau')")
    plt.xlabel("mau")
    plt.ylabel("Frequência")
    plt.show()
else:
    print("A coluna 'mau' não existe no dataframe. Verifique o nome correto das colunas.")


A coluna 'mau' não existe no dataframe. Verifique o nome correto das colunas.


#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [38]:

# Criar DataFrame de metadados
metadados = pd.DataFrame({
    "variavel": df.columns,
    "tipo": df.dtypes.values
})

# Criar lista para armazenar a quantidade de categorias por variável
qtd_categorias = []

# Percorrer todas as colunas para contar categorias (apenas para variáveis categóricas)
for col in df.columns:
    if df[col].dtype == "object" or df[col].nunique() < 20:  # Considerando variáveis com poucas categorias
        qtd_categorias.append(df[col].nunique())
    else:
        qtd_categorias.append(None)  # Para variáveis contínuas, deixamos como None

# Adicionar a coluna de quantidade de categorias ao metadados
metadados["qtd_categorias"] = qtd_categorias

# Exibir a tabela de metadados
print(metadados)


               variavel     tipo  qtd_categorias
0                    id    int64             NaN
1           code_gender   object             2.0
2          flag_own_car   object             2.0
3       flag_own_realty   object             2.0
4          cnt_children    int64            10.0
5      amt_income_total  float64             NaN
6      name_income_type   object             5.0
7   name_education_type   object             5.0
8    name_family_status   object             5.0
9     name_housing_type   object             6.0
10           days_birth    int64             NaN
11        days_employed    int64             NaN
12           flag_mobil    int64             1.0
13      flag_work_phone    int64             2.0
14           flag_phone    int64             2.0
15           flag_email    int64             2.0
16      occupation_type   object            18.0
17      cnt_fam_members  float64            11.0


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [40]:

# Identifica variáveis categóricas que precisam de dummies
variaveis_categoricas = metadados.loc[metadados["tipo"] == "object", "variavel"].tolist()

# Cria dummies e remover as variáveis originais categóricas
df_dummies = pd.get_dummies(df, columns=variaveis_categoricas, drop_first=True, dtype=int)

# Remove colunas tipo string (caso tenha sobrado alguma)
df_modelo = df_dummies.select_dtypes(exclude=['object'])

print(df_modelo.head())


        id  cnt_children  amt_income_total  days_birth  days_employed  \
0  5008804             0          427500.0      -12005          -4542   
1  5008805             0          427500.0      -12005          -4542   
2  5008806             0          112500.0      -21474          -1134   
3  5008808             0          270000.0      -19110          -3051   
4  5008809             0          270000.0      -19110          -3051   

   flag_mobil  flag_work_phone  flag_phone  flag_email  cnt_fam_members  ...  \
0           1                1           0           0              2.0  ...   
1           1                1           0           0              2.0  ...   
2           1                0           0           0              2.0  ...   
3           1                0           1           1              1.0  ...   
4           1                0           1           1              1.0  ...   

   occupation_type_Laborers  occupation_type_Low-skill Laborers  \
0            

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [42]:
# Verificar as colunas do dataframe
print(df.columns)

# Corrigir possíveis espaços extras nos nomes das colunas
df.columns = df.columns.str.strip().str.lower()  # Convertendo para minúsculas para evitar erros

# Verificar novamente se a coluna existe
if 'possui_email' in df.columns:
    cross_email = pd.crosstab(df['possui_email'], df['mau'], normalize='index') * 100
    print(cross_email)
else:
    print("A coluna 'possui_email' não existe no dataframe. Verifique o nome correto.")



Index(['id', 'code_gender', 'flag_own_car', 'flag_own_realty', 'cnt_children',
       'amt_income_total', 'name_income_type', 'name_education_type',
       'name_family_status', 'name_housing_type', 'days_birth',
       'days_employed', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members'],
      dtype='object')
A coluna 'possui_email' não existe no dataframe. Verifique o nome correto.


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [44]:
# Salvar a base transformada em CSV
df.to_csv("base.csv", index=False)
